Make sure you download the 2016-2017 Household LSMS survey data for Malawi from https://microdata.worldbank.org/index.php/catalog/lsms. It should be in `../countries/malawi_2016/LSMS/`

In [10]:
import pandas as pd
import numpy as np
import os

In [34]:
BASE_DIR = '..'
NIGHTLIGHTS_DIR = os.path.join(BASE_DIR, 'data/Nightlights/2013/F182013.v4c_web.stable_lights.avg_vis.tif')
COUNTRY = 'ethiopia_2016'

COUNTRY_DIR = os.path.join(BASE_DIR, 'countries', COUNTRY)
LSMS_DIR = os.path.join(COUNTRY_DIR, 'LSMS', 'ETH_2015_ESS_v03_M_CSV')

In [63]:
# these vary from one LSMS survey to another
CONSUMPTION_FILE = 'cons_agg_w3.csv'
CONSUMPTION_PH_COL = 'total_cons_ann' # per household
#CONSUMPTION_PC_COL = 'rexpaggpc' # per capita

GEOLOCATION_FILE = 'Geovariables/ETH_HouseholdGeovars_y3.csv'
LATITUDE_COL = 'lat_dd_mod'
LONGITUDE_COL = 'lon_dd_mod'

# purchasing power parity for malawi in 2016 (https://data.worldbank.org/indicator/PA.NUS.PRVT.PP?locations=MW)
PPP = 207.238

In [64]:
for file in [CONSUMPTION_FILE, GEOLOCATION_FILE]:
    #print(os.path.isfile(os.path.join(LSMS_DIR, file)))
    assert os.path.isfile(os.path.join(LSMS_DIR, file)), print(f'Could not find {file}')

In [65]:
df_geo = pd.read_csv(os.path.join(LSMS_DIR, GEOLOCATION_FILE))
df_geo.columns.tolist()

['household_id2',
 'ea_id2',
 'dist_road',
 'dist_popcenter',
 'dist_market',
 'dist_borderpost',
 'dist_admctr',
 'pop_density',
 'af_bio_1',
 'af_bio_8',
 'af_bio_12',
 'af_bio_13',
 'af_bio_16',
 'fsrad3_agpct',
 'fsrad3_lcmaj',
 'ssa_aez09',
 'srtm',
 'twi',
 'srtm_5_15',
 'sq1',
 'sq2',
 'sq3',
 'sq4',
 'sq5',
 'sq6',
 'sq7',
 'anntot_avg',
 'wetQ_avg',
 'wetQ_avgstart',
 'h2015_tot',
 'h2015_wetQ',
 'h2015_wetQstart',
 'eviarea_avg',
 'evimax_avg',
 'grn_avg',
 'sen_avg',
 'h2015_eviarea',
 'h2015_evimax',
 'h2015_grn',
 'h2015_sen',
 'lat_dd_mod',
 'lon_dd_mod']

In [66]:
df = pd.read_csv(os.path.join(LSMS_DIR, CONSUMPTION_FILE))
df.head()

,household_id,household_id2,ea_id,ea_id2,saq01,rural,pw_w3,adulteq,hh_size,no_conv,no_cons,food_cons_ann,nonfood_cons_ann,educ_cons_ann,total_cons_ann,price_index_hce,nom_totcons_aeq,cons_quint
0,1010101601002,10101088801601002,1010101601,10101088801601,1,1,2897.155029,0.74,1,0,0,1970.800049,1013.0,0.0,2983.800049,1.034,4032.162109,2.0
1,1010101601017,10101088801601017,1010101601,10101088801601,1,1,2897.155029,7.21,9,0,0,7883.200195,5337.0,358.0,13578.200195,1.034,1883.245483,1.0
2,1010101601034,10101088801601034,1010101601,10101088801601,1,1,2897.155029,0.74,1,0,0,8958.444458,322.0,0.0,9280.444336,1.034,12541.140625,5.0
3,1010101601049,10101088801601049,1010101601,10101088801601,1,1,2897.155029,2.50,3,0,0,9594.000000,1630.0,480.0,11704.000000,1.034,4681.600098,3.0
4,1010101601064,10101088801601064,1010101601,10101088801601,1,1,2897.155029,1.58,2,0,0,11702.888916,3272.0,0.0,14974.888672,1.034,9477.778320,5.0


In [67]:
#df = pd.read_csv(os.path.join(LSMS_DIR, CONSUMPTION_FILE))
df['cons_ph'] = df[CONSUMPTION_PH_COL]
#df['cons_pc'] = df[CONSUMPTION_PC_COL]
df['people_in_household'] = df['hh_size']
df['cons_ph'] = df['cons_ph'] / PPP / 365
df = df[['household_id2', 'cons_ph', 'people_in_household']]

df_geo = pd.read_csv(os.path.join(LSMS_DIR, GEOLOCATION_FILE))
df_cords = df_geo[['household_id2', 'ea_id2', LATITUDE_COL, LONGITUDE_COL]]
df_cords.rename(columns={LATITUDE_COL: 'lat', LONGITUDE_COL: 'lon'}, inplace=True)

In [73]:
df_geo.head()

,household_id2,ea_id2,dist_road,dist_popcenter,dist_market,dist_borderpost,dist_admctr,pop_density,af_bio_1,af_bio_8,...,eviarea_avg,evimax_avg,grn_avg,sen_avg,h2015_eviarea,h2015_evimax,h2015_grn,h2015_sen,lat_dd_mod,lon_dd_mod
0,10101088801601002,10101088801601,50,53,105,134,198,0-50,264,258,...,46,0.432,177,257,40,0.386,173,256,14.353816,37.890876
1,10101088801601017,10101088801601,50,53,106,135,198,0-50,263,257,...,46,0.432,177,257,40,0.386,173,256,14.353816,37.890876
2,10101088801601034,10101088801601,46,49,100,135,195,0-50,262,256,...,46,0.432,177,257,40,0.386,173,256,14.353816,37.890876
3,10101088801601049,10101088801601,46,49,100,135,195,0-50,262,256,...,46,0.432,177,257,40,0.386,173,256,14.353816,37.890876
4,10101088801601064,10101088801601,46,49,100,135,195,0-50,262,256,...,46,0.432,177,257,40,0.386,173,256,14.353816,37.890876


In [74]:
df_combined = pd.merge(df, df_cords, on='household_id2')

In [75]:
df_combined.head()

,household_id2,cons_ph,people_in_household,ea_id2,lat,lon
0,10101088801601002,0.039446,1,10101088801601,14.353816,37.890876
1,10101088801601017,0.179506,9,10101088801601,14.353816,37.890876
2,10101088801601034,0.122689,1,10101088801601,14.353816,37.890876
3,10101088801601049,0.154729,3,10101088801601,14.353816,37.890876
4,10101088801601064,0.197971,2,10101088801601,14.353816,37.890876


In [78]:
df_combined.drop(['household_id2', 'ea_id2'], axis=1, inplace=True)

In [79]:
df_combined.dropna(inplace=True) # can't use na values
df_combined.head()

,cons_ph,people_in_household,lat,lon
0,0.039446,1,14.353816,37.890876
1,0.179506,9,14.353816,37.890876
2,0.122689,1,14.353816,37.890876
3,0.154729,3,14.353816,37.890876
4,0.197971,2,14.353816,37.890876


In [80]:
df_combined.shape

(4717, 4)

Jean et al makes a minor mistake in computing per capita consumption. Even though the LSMS documentation says `rexpagg` is per capita, `rexpaggpc` is actually the per capita value. `rexpagg` is the household consumption. We can compute per capita consumption in a cluster by summing up `rexpagg` in a cluster and dividing by the number of people in the cluster (that were surveyed).

In [81]:
df_clusters = df_combined.groupby(['lat', 'lon']).sum().reset_index()
df_clusters['cons_pc'] = df_clusters['cons_ph'] / df_clusters['people_in_household']
df_clusters.drop(['cons_ph', 'people_in_household'], axis=1, inplace=True)

In [82]:
df_clusters.head()

,lat,lon,cons_pc
0,3.455701,39.515994,0.079805
1,3.549937,39.184234,0.067952
2,3.864243,39.101366,0.059437
3,3.982931,38.491368,0.057343
4,4.048194,41.930928,0.045284


In [83]:
df_clusters.shape

(523, 3)

This shows how we can use the downloaded nightlight data to get the nightlights value at these cluster points.

In [84]:
import geoio
img = geoio.GeoImage(NIGHTLIGHTS_DIR)
# pass lon then lat
xPixel, yPixel = img.proj_to_raster(34.915074, -14.683761)

In [85]:
xPixel, yPixel

(25790.308983159237, 10762.551363048204)

In [86]:
img.get_data().shape

(1, 16801, 43201)

In [87]:
im_array = np.squeeze(img.get_data())
im_array.shape

(16801, 43201)

In [88]:
im_array[int(yPixel),int(xPixel)] # this is the nightlight value at the given coordinate

0

In [89]:
import math

def create_space(lat, lon, s=10):
    """Creates a s km x s km square centered on (lat, lon)"""
    v = (180/math.pi)*(500/6378137)*s # roughly 0.045 for s=10
    return lat - v, lon - v, lat + v, lon + v

In [90]:
cluster_nightlights = []
for i,r in df_clusters.iterrows():
    min_lat, min_lon, max_lat, max_lon = create_space(r.lat, r.lon)
    xminPixel, yminPixel = img.proj_to_raster(min_lon, min_lat)
    xmaxPixel, ymaxPixel = img.proj_to_raster(max_lon, max_lat)
    
    xminPixel, xmaxPixel = min(xminPixel, xmaxPixel), max(xminPixel, xmaxPixel)
    yminPixel, ymaxPixel = min(yminPixel, ymaxPixel), max(yminPixel, ymaxPixel)
    
    xminPixel, yminPixel, xmaxPixel, ymaxPixel = int(xminPixel), int(yminPixel), int(xmaxPixel), int(ymaxPixel)
    cluster_nightlights.append(im_array[yminPixel:ymaxPixel,xminPixel:xmaxPixel].mean())

In [91]:
df_clusters['nightlights'] = cluster_nightlights

In [95]:
sum(df_clusters['nightlights'] >0)

187

In [92]:
df_clusters.head()

,lat,lon,cons_pc,nightlights
0,3.455701,39.515994,0.079805,0.0
1,3.549937,39.184234,0.067952,0.0
2,3.864243,39.101366,0.059437,0.0
3,3.982931,38.491368,0.057343,0.0
4,4.048194,41.930928,0.045284,0.0


In [93]:
os.makedirs(os.path.join(COUNTRY_DIR, 'processed'), exist_ok=True)

In [94]:
df_clusters.to_csv(os.path.join(COUNTRY_DIR, 'processed/clusters.csv'), index=False)